<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/assignment2/notebooks/Augmentation_wip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torchvision==0.8.2  -q -q -q
!pip install torchtext==0.8.1    -q -q -q
!pip install torchaudio==0.7.2   -q -q -q
!pip install torch==1.7.1        -q -q -q
!pip install tqdm==4.49.0        -q -q -q
!pip install transformers==4.2.2 -q -q -q
!pip install tensorflow          -q -q -q
!pip install tensorboard         -q -q -q
!pip install tensorboardX        -q -q -q
!pip install --upgrade virtualenv -q -q -q
!pip install sentencepiece==0.1.94       -q -q -q
!pip install mosestokenizer==1.1.0 -q -q -q

!rm -rf mtu-nlp-assignment
!git clone https://github.com/bhattacharjee/mtu-nlp-assignment.git

Cloning into 'mtu-nlp-assignment'...
remote: Enumerating objects: 1006, done.
remote: Counting objects: 100% (1006/1006), done.
remote: Compressing objects: 100% (871/871), done.
remote: Total 1006 (delta 492), reused 383 (delta 131), pack-reused 0
Receiving objects: 100% (1006/1006), 96.08 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (492/492), done.


In [2]:
# Taken from https://amitness.com/back-translation/

from transformers import MarianMTModel, MarianTokenizer

target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts, return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts


def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    if isinstance(texts, str):
        texts = [texts]
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts



FileNotFoundError: ignored